In [3]:
import mysql.connector
import tkinter as tk
from tkinter import ttk

# Database functions
def initialize_connection():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="feb.16@mysql",
        database="sales"
    )

    cursor = conn.cursor()
    create_users_table(cursor)
    create_sales_table(cursor)

    return conn, cursor

def create_users_table(cursor):
    cursor.execute("SHOW TABLES")
    temp = cursor.fetchall()
    tables = [item[0] for item in temp]

    if "users" not in tables:
        cursor.execute("""CREATE TABLE users(
            id INT AUTO_INCREMENT PRIMARY KEY,
            firstName VARCHAR(100),
            lastName VARCHAR(100),
            email VARCHAR(100) UNIQUE,
            password VARCHAR(30)
         )""")

def create_sales_table(cursor):
    cursor.execute("SHOW TABLES")
    temp = cursor.fetchall()
    tables = [item[0] for item in temp]

    if "sales" not in tables:
        cursor.execute("""CREATE TABLE sales(
            id INT AUTO_INCREMENT PRIMARY KEY,
            product_name VARCHAR(100),
            product_category VARCHAR(100),
            quantity INT,
            discount DECIMAL(10, 2),
            tax DECIMAL(10, 2),
            price DECIMAL(10, 2),
            sales_date DATE
         )""")

def login(cursor, data):
    cursor.execute(f"""SELECT * FROM users WHERE email = '{data["email"]}' 
                       AND password = '{data["password"]}' """)

    if cursor.fetchone() is not None:
        return True
    return False

def register(cursor, conn, data):
    cursor.execute(f"""INSERT INTO users (firstName, lastName, email, password) 
                      VALUES ('{data["firstName"]}', '{data["lastName"]}', 
                      '{data["email"]}', '{data["password"]}')""")
    conn.commit()

def insert_sale(cursor, conn, data):
    cursor.execute("""INSERT INTO sales (product_name, product_category, quantity, discount, tax, price, sales_date) 
                      VALUES (%s, %s, %s, %s, %s, %s, %s)""",
                   (data["product_name"], data["product_category"], data["quantity"], data["discount"], data["tax"], data["price"], data["sales_date"]))
    conn.commit()

def view_sales(cursor):
    cursor.execute("SELECT * FROM sales")
    return cursor.fetchall()

def edit_sale(cursor, conn, sale_id, data):
    cursor.execute("""UPDATE sales SET product_name=%s, product_category=%s, quantity=%s, discount=%s, tax=%s, price=%s, sales_date=%s
                      WHERE id=%s""",
                   (data["product_name"], data["product_category"], data["quantity"], data["discount"], data["tax"], data["price"], data["sales_date"], sale_id))
    conn.commit()

def delete_sale(cursor, conn, sale_id):
    cursor.execute("DELETE FROM sales WHERE id=%s", (sale_id,))
    conn.commit()

# Login and Register Windows
class LoginWindow(tk.Toplevel):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.title("Login")
        self.geometry("300x150")
        
        tk.Label(self, text="Email:").grid(row=0, column=0, padx=10, pady=5)
        self.email_entry = tk.Entry(self)
        self.email_entry.grid(row=0, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Password:").grid(row=1, column=0, padx=10, pady=5)
        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.grid(row=1, column=1, padx=10, pady=5)
        
        login_button = tk.Button(self, text="Login", command=self.login)
        login_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

    def login(self):
        data = {
            "email": self.email_entry.get(),
            "password": self.password_entry.get()
        }
        if login(cursor, data):
            print("Login successful")
            self.destroy()
            app.open_sales_window()
        else:
            print("Login failed")

class RegisterWindow(tk.Toplevel):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.title("Register")
        self.geometry("400x300")
        
        tk.Label(self, text="First Name:").grid(row=0, column=0, padx=10, pady=5)
        self.first_name_entry = tk.Entry(self)
        self.first_name_entry.grid(row=0, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Last Name:").grid(row=1, column=0, padx=10, pady=5)
        self.last_name_entry = tk.Entry(self)
        self.last_name_entry.grid(row=1, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Email:").grid(row=2, column=0, padx=10, pady=5)
        self.email_entry = tk.Entry(self)
        self.email_entry.grid(row=2, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Password:").grid(row=3, column=0, padx=10, pady=5)
        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.grid(row=3, column=1, padx=10, pady=5)
        
        register_button = tk.Button(self, text="Register", command=self.register)
        register_button.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

    def register(self):
        data = {
            "firstName": self.first_name_entry.get(),
            "lastName": self.last_name_entry.get(),
            "email": self.email_entry.get(),
            "password": self.password_entry.get()
        }
        register(cursor, conn, data)
        print("Registration successful")
        self.destroy()

# Sales Management System Windows
class SalesDataWindow(tk.Toplevel):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.title("Sales Management")
        self.geometry("400x150")

        self.create_widgets()

    def create_widgets(self):
        add_sales_button = tk.Button(self, text="Add Sales Data", command=self.open_add_sales_window)
        add_sales_button.pack(pady=10)

        view_sales_button = tk.Button(self, text="View Sales Data", command=self.open_view_sales_window)
        view_sales_button.pack(pady=10)

    def open_add_sales_window(self):
        AddSalesWindow(self)

    def open_view_sales_window(self):
        ViewSalesWindow(self)

class AddSalesWindow(tk.Toplevel):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.title("Add Sales Data")
        self.geometry("400x300")

        self.create_widgets()

    def create_widgets(self):
        # GUI elements for entering sales data
        tk.Label(self, text="Product Name:").grid(row=0, column=0, padx=10, pady=5)
        self.product_name_entry = tk.Entry(self)
        self.product_name_entry.grid(row=0, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Product Category:").grid(row=1, column=0, padx=10, pady=5)
        self.product_category_entry = tk.Entry(self)
        self.product_category_entry.grid(row=1, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Quantity:").grid(row=2, column=0, padx=10, pady=5)
        self.quantity_entry = tk.Entry(self)
        self.quantity_entry.grid(row=2, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Discount:").grid(row=3, column=0, padx=10, pady=5)
        self.discount_entry = tk.Entry(self)
        self.discount_entry.grid(row=3, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Tax:").grid(row=4, column=0, padx=10, pady=5)
        self.tax_entry = tk.Entry(self)
        self.tax_entry.grid(row=4, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Price:").grid(row=5, column=0, padx=10, pady=5)
        self.price_entry = tk.Entry(self)
        self.price_entry.grid(row=5, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Sales Date (YYYY-MM-DD):").grid(row=6, column=0, padx=10, pady=5)
        self.sales_date_entry = tk.Entry(self)
        self.sales_date_entry.grid(row=6, column=1, padx=10, pady=5)

        submit_button = tk.Button(self, text="Submit", command=self.submit)
        submit_button.grid(row=7, column=1, padx=10, pady=10)

    def submit(self):
        data = {
            "product_name": self.product_name_entry.get(),
            "product_category": self.product_category_entry.get(),
            "quantity": int(self.quantity_entry.get()),
            "discount": float(self.discount_entry.get()),
            "tax": float(self.tax_entry.get()),
            "price": float(self.price_entry.get()),
            "sales_date": self.sales_date_entry.get()
        }

        insert_sale(cursor, conn, data)
        self.product_name_entry.delete(0, tk.END)
        self.product_category_entry.delete(0, tk.END)
        self.quantity_entry.delete(0, tk.END)
        self.discount_entry.delete(0, tk.END)
        self.tax_entry.delete(0, tk.END)
        self.price_entry.delete(0, tk.END)
        self.sales_date_entry.delete(0, tk.END)
        print("Sales data added successfully")

class ViewSalesWindow(tk.Toplevel):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.title("View Sales Data")
        self.geometry("800x400")

        self.create_widgets()

    def create_widgets(self):
        # GUI elements for viewing sales data
        self.tree = ttk.Treeview(self, columns=("ID", "Product Name", "Product Category", "Quantity", "Discount", "Tax", "Price", "Sales Date"), show="headings")
        self.tree.heading("ID", text="ID")
        self.tree.heading("Product Name", text="Product Name")
        self.tree.heading("Product Category", text="Product Category")
        self.tree.heading("Quantity", text="Quantity")
        self.tree.heading("Discount", text="Discount")
        self.tree.heading("Tax", text="Tax")
        self.tree.heading("Price", text="Price")
        self.tree.heading("Sales Date", text="Sales Date")
        self.tree.pack(fill="both", expand=True)

        self.populate_tree()

        scroll = ttk.Scrollbar(self, orient="vertical", command=self.tree.yview)
        scroll.pack(side="right", fill="y")
        self.tree.configure(yscrollcommand=scroll.set)

        edit_button = tk.Button(self, text="Edit Record", command=self.edit_record)
        edit_button.pack(side="left", padx=10, pady=10)

        delete_button = tk.Button(self, text="Delete Record", command=self.delete_record)
        delete_button.pack(side="left", padx=10, pady=10)

    def populate_tree(self):
        sales_data = view_sales(cursor)
        if sales_data:
            for row in sales_data:
                self.tree.insert("", "end", values=row)

    def edit_record(self):
        selected_item = self.tree.selection()
        if not selected_item:
            return

        sale_id = self.tree.item(selected_item, "values")[0]
        EditSalesWindow(self, sale_id)

    def delete_record(self):
        selected_item = self.tree.selection()
        if not selected_item:
            return

        sale_id = self.tree.item(selected_item, "values")[0]
        delete_sale(cursor, conn, sale_id)
        self.tree.delete(selected_item)

class EditSalesWindow(tk.Toplevel):
    def __init__(self, master, sale_id):
        super().__init__(master)
        self.master = master
        self.title("Edit Sales Record")
        self.geometry("400x400")

        self.sale_id = sale_id
        self.fetch_data()

    def fetch_data(self):
        cursor.execute("SELECT * FROM sales WHERE id=%s", (self.sale_id,))
        sale_data = cursor.fetchone()

        self.product_name_var = tk.StringVar(value=sale_data[1])
        self.product_category_var = tk.StringVar(value=sale_data[2])
        self.quantity_var = tk.StringVar(value=sale_data[3])
        self.discount_var = tk.StringVar(value=sale_data[4])
        self.tax_var = tk.StringVar(value=sale_data[5])
        self.price_var = tk.StringVar(value=sale_data[6])
        self.sales_date_var = tk.StringVar(value=sale_data[7])

        self.create_widgets()

    def create_widgets(self):
        tk.Label(self, text="Product Name:").grid(row=0, column=0, padx=10, pady=5)
        self.product_name_entry = tk.Entry(self, textvariable=self.product_name_var)
        self.product_name_entry.grid(row=0, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Product Category:").grid(row=1, column=0, padx=10, pady=5)
        self.product_category_entry = tk.Entry(self, textvariable=self.product_category_var)
        self.product_category_entry.grid(row=1, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Quantity:").grid(row=2, column=0, padx=10, pady=5)
        self.quantity_entry = tk.Entry(self, textvariable=self.quantity_var)
        self.quantity_entry.grid(row=2, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Discount:").grid(row=3, column=0, padx=10, pady=5)
        self.discount_entry = tk.Entry(self, textvariable=self.discount_var)
        self.discount_entry.grid(row=3, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Tax:").grid(row=4, column=0, padx=10, pady=5)
        self.tax_entry = tk.Entry(self, textvariable=self.tax_var)
        self.tax_entry.grid(row=4, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Price:").grid(row=5, column=0, padx=10, pady=5)
        self.price_entry = tk.Entry(self, textvariable=self.price_var)
        self.price_entry.grid(row=5, column=1, padx=10, pady=5)
        
        tk.Label(self, text="Sales Date (YYYY-MM-DD):").grid(row=6, column=0, padx=10, pady=5)
        self.sales_date_entry = tk.Entry(self, textvariable=self.sales_date_var)
        self.sales_date_entry.grid(row=6, column=1, padx=10, pady=5)

        submit_button = tk.Button(self, text="Update", command=self.update_record)
        submit_button.grid(row=7, column=1, padx=10, pady=10)

    def update_record(self):
        data = {
            "product_name": self.product_name_var.get(),
            "product_category": self.product_category_var.get(),
            "quantity": int(self.quantity_var.get()),
            "discount": float(self.discount_var.get()),
            "tax": float(self.tax_var.get()),
            "price": float(self.price_var.get()),
            "sales_date": self.sales_date_var.get()
        }

        edit_sale(cursor, conn, self.sale_id, data)
        self.destroy()

# Main Application Window
class MainWindow(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Sales Management System")
        self.geometry("400x200")

        self.initialize_db()
        self.create_widgets()

    def initialize_db(self):
        global conn, cursor
        conn, cursor = initialize_connection()

    def create_widgets(self):
        login_button = tk.Button(self, text="Login", command=self.open_login_window)
        login_button.pack(pady=10)

        register_button = tk.Button(self, text="Register", command=self.open_register_window)
        register_button.pack(pady=10)

    def open_login_window(self):
        LoginWindow(self)

    def open_register_window(self):
        RegisterWindow(self)

    def open_sales_window(self):
        SalesDataWindow(self)

if __name__ == "__main__":
    app = MainWindow()
    app.mainloop()


Login successful
Login successful
Sales data added successfully
